In [1]:
import pandas as pd

import json

from copy import deepcopy

___
Initial exploration and helper functions

In [4]:
arxiv = pd.read_json('arxiv-metadata-oai-snapshot.json', lines=True, nrows=10)

MemoryError: 

In [8]:
with open('tmp.txt') as f: 
    tmp = json.loads(f.read())

In [10]:
tmp.keys()

dict_keys(['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'abstract', 'versions', 'update_date', 'authors_parsed'])

In [81]:
def get_line_count(filename):
    counter = 0
    with open(filename) as f: 
        for line in f:
            counter +=1
    return counter

In [83]:
get_line_count('arxiv-metadata-oai-snapshot.json')

1796911

In [86]:
1800000/1000*0.6

1080.0

In [17]:
def read_n_entries(filename,n):
    with open(filename) as f: 
        head = [json.loads(next(f)) for x in range(n)];
    return head

In [73]:
def parse_json(entry): 
    keys = ['id','authors','title','categories','versions','authors_parsed','doi']
    keys_binary = ['journal-ref','doi','report-no']
    out = {k: deepcopy(entry[k]) for k in keys}
    out['is_published'] = any([(entry[k] != None) for k in keys_binary])
    return out

In [74]:
def read_parse_n_entries(filename,n):
    with open(filename) as f: 
        head = [parse_json(json.loads(next(f))) for x in range(n)];
    return head

In [89]:
def read_parse_all_entries_df(filename):
    out = []
    with open(filename) as f: 
        for line in f:
            out.append(parse_json(json.loads(line)))
    return pd.DataFrame(out)

In [63]:
tmpj = read_n_entries('arxiv-metadata-oai-snapshot.json',10)

In [79]:
pd.DataFrame(read_parse_n_entries('arxiv-metadata-oai-snapshot.json',1000)).info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1000 non-null   object
 1   authors         1000 non-null   object
 2   title           1000 non-null   object
 3   categories      1000 non-null   object
 4   versions        1000 non-null   object
 5   authors_parsed  1000 non-null   object
 6   doi             659 non-null    object
 7   is_published    1000 non-null   bool  
dtypes: bool(1), object(7)
memory usage: 596.4 KB


In [80]:
pd.DataFrame(read_n_entries('arxiv-metadata-oai-snapshot.json',1000)).info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1000 non-null   object
 1   submitter       1000 non-null   object
 2   authors         1000 non-null   object
 3   title           1000 non-null   object
 4   comments        901 non-null    object
 5   journal-ref     573 non-null    object
 6   doi             659 non-null    object
 7   report-no       78 non-null     object
 8   categories      1000 non-null   object
 9   license         58 non-null     object
 10  abstract        1000 non-null   object
 11  versions        1000 non-null   object
 12  update_date     1000 non-null   object
 13  authors_parsed  1000 non-null   object
dtypes: object(14)
memory usage: 1.8 MB


In [62]:
df = pd.DataFrame(read_n_entries('arxiv-metadata-oai-snapshot.json',10))

___

## Read the whole parsed json

In [90]:
%%time
arxiv = read_parse_all_entries_df('arxiv-metadata-oai-snapshot.json')

Wall time: 2min 16s


In [91]:
arxiv.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1796911 entries, 0 to 1796910
Data columns (total 8 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   id              object
 1   authors         object
 2   title           object
 3   categories      object
 4   versions        object
 5   authors_parsed  object
 6   doi             object
 7   is_published    bool  
dtypes: bool(1), object(7)
memory usage: 1.1 GB


Categories statistics

In [92]:
arxiv['categories'].value_counts()

astro-ph                                         86914
hep-ph                                           73550
quant-ph                                         53966
hep-th                                           53287
cond-mat.mtrl-sci                                30107
                                                 ...  
physics.class-ph cond-mat.mtrl-sci gr-qc             1
cs.CG math.GT math.MG                                1
cs.CL cs.SI q-fin.ST                                 1
stat.ML cs.LG math.OC math.ST stat.AP stat.TH        1
math.AG cs.SC math.CV                                1
Name: categories, Length: 62055, dtype: int64

In [94]:
arxiv['categories'].apply(lambda x: len(x.split(' '))).value_counts()

1     1025259
2      502711
3      187804
4       60999
5       16025
6        3404
7         541
8         121
9          30
10         14
11          2
13          1
Name: categories, dtype: int64

So we have ca. 2/3 entries with a single classification, and rest belongs to more than 1 category

Let's look at the extreme side:

In [95]:
arxiv['categories'].apply(lambda x: len(x.split(' '))).sort_values(ascending=False).head(20)

77549      13
57365      11
1545826    11
1777614    10
1688950    10
77796      10
1677040    10
1463715    10
1638391    10
1796672    10
1772146    10
19444      10
18505      10
141660     10
1543928    10
1675693    10
1736009    10
1531230     9
1383087     9
158233      9
Name: categories, dtype: int64

In [101]:
arxiv['n_categories'] = arxiv['categories'].apply(lambda x: len(x.split(' ')))

In [97]:
arxiv.iloc[77549]['title']

'The finite harmonic oscillator and its associated sequences'

In [98]:
arxiv.iloc[57365]['title']

'Visco-potential free-surface flows and long wave modelling'

In [100]:
arxiv.iloc[1545826]

id                                                       cs/0410017
authors                    Carl S. McTague and James P. Crutchfield
title             Automated Pattern Detection--An Algorithm for ...
categories        cs.CV cond-mat.stat-mech cs.CL cs.DS cs.IR cs....
versions          [{'version': 'v1', 'created': 'Thu, 7 Oct 2004...
authors_parsed    [[McTague, Carl S., ], [Crutchfield, James P., ]]
doi                                                            None
is_published                                                   True
Name: 1545826, dtype: object

Look at the single categories

In [103]:
arxiv.query('n_categories == 1')['categories'].unique()

array(['hep-ph', 'physics.gen-ph', 'math.CO', 'cond-mat.mes-hall',
       'gr-qc', 'cond-mat.mtrl-sci', 'astro-ph', 'math.NT', 'hep-th',
       'hep-ex', 'math.NA', 'nlin.PS', 'math.RA', 'cond-mat.str-el',
       'physics.pop-ph', 'nucl-th', 'math.FA', 'cs.DS', 'math.DS',
       'physics.soc-ph', 'math.AG', 'math.OA', 'math.PR', 'math.DG',
       'physics.optics', 'math.GR', 'nlin.SI', 'math.SG',
       'physics.data-an', 'cs.CC', 'math.GT', 'quant-ph',
       'cond-mat.other', 'math.CV', 'math.AP', 'cond-mat.supr-con',
       'math.RT', 'cond-mat.stat-mech', 'q-bio.OT', 'physics.plasm-ph',
       'nlin.CG', 'nucl-ex', 'cond-mat.soft', 'physics.comp-ph',
       'math.MG', 'math.QA', 'physics.bio-ph', 'physics.chem-ph',
       'math.AT', 'physics.geo-ph', 'q-bio.BM', 'math.OC', 'cs.CR',
       'physics.class-ph', 'q-bio.PE', 'q-bio.NC', 'physics.atom-ph',
       'math.GM', 'hep-lat', 'math.CA', 'physics.atm-clus', 'cs.PF',
       'physics.acc-ph', 'math.SP', 'nlin.CD', 'physics.hist-ph'

In [108]:
versions = arxiv.iloc[0]['versions']

In [109]:
versions

[{'version': 'v1', 'created': 'Mon, 2 Apr 2007 19:18:42 GMT'},
 {'version': 'v2', 'created': 'Tue, 24 Jul 2007 20:10:27 GMT'}]

In [154]:
def extract_dt_submission(versions_list):
    tmp = pd.to_datetime([item['created'] for item in versions_list if item['version']=='v1'][0])    
    return tmp

In [156]:
arxiv.head(100)['versions'].apply(extract_dt_submission).memory_usage(deep=True)

928

In [160]:
%%time
arxiv['submitted'] = arxiv['versions'].apply(extract_dt_submission)

Wall time: 11min 25s


In [161]:
arxiv.head()['submitted']

0   2007-04-02 19:18:42+00:00
1   2007-03-31 02:26:18+00:00
2   2007-04-01 20:46:54+00:00
3   2007-03-31 03:16:14+00:00
4   2007-04-02 18:09:58+00:00
Name: submitted, dtype: datetime64[ns, UTC]

In [162]:
arxiv.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1796911 entries, 0 to 1796910
Data columns (total 10 columns):
 #   Column          Dtype              
---  ------          -----              
 0   id              object             
 1   authors         object             
 2   title           object             
 3   categories      object             
 4   versions        object             
 5   authors_parsed  object             
 6   doi             object             
 7   is_published    bool               
 8   n_categories    int64              
 9   submitted       datetime64[ns, UTC]
dtypes: bool(1), datetime64[ns, UTC](1), int64(1), object(7)
memory usage: 1.1 GB


In [164]:
%%time
arxiv['n_versions'] = arxiv['versions'].apply(lambda x: len(x))

Wall time: 1.01 s


In [165]:
arxiv.head()

,id,authors,title,categories,versions,authors_parsed,doi,is_published,n_categories,submitted,n_versions
0,0704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,hep-ph,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,...",10.1103/PhysRevD.76.013009,True,1,2007-04-02 19:18:42+00:00,2
1,0704.0002,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,math.CO cs.CG,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...","[[Streinu, Ileana, ], [Theran, Louis, ]]",None,False,2,2007-03-31 02:26:18+00:00,2
2,0704.0003,Hongjun Pan,The evolution of the Earth-Moon system based o...,physics.gen-ph,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...","[[Pan, Hongjun, ]]",None,False,1,2007-04-01 20:46:54+00:00,3
3,0704.0004,David Callan,A determinant of Stirling cycle numbers counts...,math.CO,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...","[[Callan, David, ]]",None,False,1,2007-03-31 03:16:14+00:00,1
4,0704.0005,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,math.CA math.FA,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...","[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]",None,True,2,2007-04-02 18:09:58+00:00,1


In [185]:
%%time
arxiv.to_pickle('arxiv.pickle')

Wall time: 29.9 s
